In [43]:
# import
import csv
import math
import random
import sys
import numpy as np
import matplotlib.pyplot as plt
from pystruct.models import ChainCRF
from pystruct.learners import FrankWolfeSSVM

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import (RBF, Matern, RationalQuadratic,
                                              ExpSineSquared, DotProduct,
                                              ConstantKernel, WhiteKernel)

%matplotlib inline

random.seed

<bound method Random.seed of <random.Random object at 0x10287ea18>>

In [55]:
kernel = 1.0 * RationalQuadratic(length_scale=0.999, alpha=136.)
# for floor in range(0,5):
#     for building in range(0,3):
for floor in range(2,3):
    for building in range(0,1):
        
        ## Count the number of data points in building id & floor id
        data_num = 0
        with open("1478167720_9233432_trainingData.csv", newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=',')
            for row in spamreader:
                if (row[523] == 'BUILDINGID'):
                    continue
                elif (int(row[523]) is not building or int(row[522]) is not floor):
                    continue
                data_num += 1
        print(data_num)
        ## if there are no data, continue to next floor 
        if (data_num == 0):
            continue
            
        ## Load data points in
        wifi_loc_time = np.zeros(shape = (data_num, 524))
        i=-1
        with open("1478167720_9233432_trainingData.csv", newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=',')
            for row in spamreader:
                if (row[523] == 'BUILDINGID'):
                    continue
                elif (int(row[523]) is not building or int(row[522]) is not floor):
                    continue
                i = i+1
                if (i > data_num):
                    break
                # wifi
                wifi_loc_time[i-1][:520] = np.array(row[:520])
                # location x, y
                wifi_loc_time[i-1][520:522] = np.array(row[520:522])
                # userID
                wifi_loc_time[i-1][522] = np.array(row[526])
                # time stamp
                wifi_loc_time[i-1][-1] = np.array(row[-1])
        
        ## Sort by time stamp
        wifi_loc_time = wifi_loc_time[wifi_loc_time[:,-1].argsort()]
        
        ## List all users
        user_list = np.unique(wifi_loc_time[:, 522])
        user_num = len(user_list)
        print("On floor ", floor, " in building ", building, "there are ", user_num, " users.")
        
        ## Make grids
        grid_slice = 70
                                    ## max boundary , grid size, min boundary
        longitude_list = np.array([max(wifi_loc_time[:, 520]), (max(wifi_loc_time[:, 520])-min(wifi_loc_time[:, 520]))/grid_slice\
                                   , min(wifi_loc_time[:, 520])])
        latitude_list = np.array([max(wifi_loc_time[:, 521]), (max(wifi_loc_time[:, 521])-min(wifi_loc_time[:, 521]))/grid_slice\
                                   , min(wifi_loc_time[:, 521])])
        
        ## state tag = [grid_slice x grid_slice]
        state_tag = np.zeros(shape=(grid_slice, grid_slice), dtype=int)
        ## initialize to -1
        for i in range(grid_slice):
            for j in range(grid_slice):
                state_tag[i][j] = -1
        
        ## Record each user's number of location logs
        user_walking_num = np.zeros(shape=(user_num), dtype=int)
        for i in range(len(wifi_loc_time[:,522])):
            for user_index in range(user_num):
                if (wifi_loc_time[i, 522] == user_list[user_index]):
                    user_walking_num[user_index] += 1
        
        ## set the sequence length of observed walking tracks
        seq_window = 9
        total_batch_num = 0
        
        ## compute all users' walking sequences and add them as number of batches
        for num in user_walking_num:
            total_batch_num += math.floor(num/seq_window)
        print("total_batch_num: ",total_batch_num)
        
        batch_wifi = np.zeros((total_batch_num, seq_window, 520))
        batch_loc = np.zeros((total_batch_num, seq_window), dtype = int)
        
        ## tracking parameters
        state_count = 0
        ## cumulative graduation [# of batch of user1] + [# of batch of user2] [...]
        batch_num = 0
        iter_batch = 0
        iter_window = 0
        state_map_grid = {}
        
        for k in range(len(user_list)):
            print("User",k, " walking ", user_walking_num[k])
            
            ## [longitude, latitude, wifi]
            user_fp = np.zeros((user_walking_num[k], 2+520))
            batch_num += math.floor(user_walking_num[k]/seq_window)
            
            ## pick out the corresponding user's data
            user_i = 0
            for i in range(len(wifi_loc_time[:,522])):
                if (wifi_loc_time[i, 522] == user_list[k]):
                    user_fp[user_i, :2] = np.array(wifi_loc_time[i, 520:522])
                    user_fp[user_i, 2:] = np.array(wifi_loc_time[i, :520])
                    user_i += 1
        
            for s in range(math.floor(user_walking_num[k]/seq_window) * seq_window):
                ## put state id
                i = int(math.ceil((user_fp[s][0] - longitude_list[2])/longitude_list[1]) - 1)
                j = int(math.ceil((user_fp[s][1] - latitude_list[2])/latitude_list[1]) - 1)
                if (i == -1):
                    i = 0
                if (j == -1):
                    j = 0
                if (state_tag[i][j] == -1):
                    ## apply GPs
                    gp = GaussianProcessRegressor(kernel=kernel,
                                          alpha=0.5)
                    grid_wifi_data = user_fp[s][2:].copy()
                    grid_loc_data = np.array([user_fp[s][0] - (longitude_list[1]*(float(i)+0.5) + longitude_list[2])\
                                              , user_fp[s][1] - (latitude_list[1]*(float(j)+0.5) + latitude_list[2])])
                    
                    state_tag[i][j] = state_count
                    state_map_grid[state_count] = [i,j, gp, grid_wifi_data, grid_loc_data]
                    print(i, j, state_count)
                    state_count += 1
                else:
                    print(i, j, state_map_grid[state_tag[i][j]][3].shape)
                    state_map_grid[state_tag[i][j]][3] = np.vstack((state_map_grid[state_tag[i][j]][3], user_fp[s][2:]))
                    state_map_grid[state_tag[i][j]][4] = np.vstack((state_map_grid[state_tag[i][j]][4], \
                               np.array([user_fp[s][0] - (longitude_list[1]*(float(i)+0.5) + longitude_list[2])\
                                        , user_fp[s][1] - (latitude_list[1]*(float(j)+0.5) + latitude_list[2])])))
                    print(state_map_grid[state_tag[i][j]][3].shape)
                    
                batch_wifi[iter_batch][iter_window] = user_fp[s][2:]
                batch_loc[iter_batch,iter_window] = state_tag[i][j]

                
                if (iter_window == (seq_window-1)):
                    iter_window = 0
                    iter_batch = iter_batch + 1
                    if (iter_batch == (batch_num)):
                        break
                else:
                    iter_window = iter_window + 1
        print("Grid is ", grid_slice, "x", grid_slice, " ; the # of states is ", state_count)
        for s in range(state_count):
            print(state_map_grid[s][3].shape, state_map_grid[s][4].shape)
            state_map_grid[s][2] = state_map_grid[s][2].fit(state_map_grid[s][3], state_map_grid[s][4])
                    
        # Testing
        # extract data
        print("Validation...")
        data_num_v = 0
        with open("1478167721_0345678_validationData.csv", newline='') as v_csvfile:
            spamreader_v = csv.reader(v_csvfile, delimiter=',')
            for row in spamreader_v:
                if (row[523] == 'BUILDINGID'):
                    continue
                elif (int(row[523]) is not building or int(row[522]) is not floor):
                    continue
                data_num_v += 1

        print(data_num_v)
        if (data_num_v == 0):
            continue
        i = -1
        wifi_loc_time_v = np.zeros(shape = (data_num_v, 524))
        
        with open("1478167721_0345678_validationData.csv", newline='') as v_csvfile:
            spamreader_v = csv.reader(v_csvfile, delimiter=',')
            for row in spamreader_v:
                if (row[523] == 'BUILDINGID'):
                    continue
                elif (int(row[523]) is not building or int(row[522]) is not floor):
                    continue
                i += 1
                #if (i > data_num):
                #    break
                # wifi
                wifi_loc_time_v[i-1][:520] = np.array(row[:520])
                # location x, y
                wifi_loc_time_v[i-1][520:522] = np.array(row[520:522])
                # userID
                wifi_loc_time_v[i-1][522] = np.array(row[526])
                # time stamp
                wifi_loc_time_v[i-1][-1] = np.array(row[-1])
                
        wifi_loc_time_v = wifi_loc_time_v[wifi_loc_time_v[:,-1].argsort()]  
                
        batch_num = int(data_num_v/seq_window)
        print("Validating data ", batch_num)
        batch_wifi_v = np.zeros((batch_num + 1, seq_window, 520))
        batch_loc_v = np.zeros((batch_num + 1, seq_window), dtype = int)

        iter_window = 0
        iter_batch = 1
        
        map_distance = 0
        adjust_grid_num = 0

        for s in range(len(wifi_loc_time_v)):
            
            i = int(math.ceil((wifi_loc_time_v[s][520]-longitude_list[2])/longitude_list[1]) - 1)
            j = int(math.ceil((wifi_loc_time_v[s][521]-latitude_list[2])/latitude_list[1]) - 1)
            if (i < -1):
                i = 0
            elif (i >= grid_slice): 
                i = grid_slice-1
            if (j < 0):
                j = 0
            elif (j >= grid_slice): 
                j = grid_slice-1
            if (state_tag[i][j] == -1):
                adjust_grid_num+=1
                print(i, j, state_tag[i][j])
                min_dist = np.zeros(shape=(state_count, 1))
                for x in range(state_count):
                    min_dist[x,0] = math.sqrt(pow((float(state_map_grid[x][0])+0.5)*longitude_list[1]+longitude_list[2] - wifi_loc_time_v[s][520],2)+\
                              pow((float(state_map_grid[x][1])+0.5)*latitude_list[1]+latitude_list[2] - wifi_loc_time_v[s][521],2))
                map_distance += np.min(min_dist)
                state_tag[i][j] = np.argmin(min_dist)
            
            batch_wifi_v[iter_batch][iter_window] = wifi_loc_time_v[s][:520]
            batch_loc_v[iter_batch,iter_window] = state_tag[i][j]
            if (iter_window == (seq_window-1)):
                iter_window = 0
                iter_batch = iter_batch + 1
                if (iter_batch == (batch_num)):
                    break
            else:
                iter_window = iter_window + 1  
        print("Grids need adjustment: ", adjust_grid_num)
        print("Base map distance is ", map_distance/data_num_v)
                
        model = ChainCRF()
        ssvm = FrankWolfeSSVM(model=model, C=.1, max_iter=80)            
        ssvm.fit(batch_wifi, batch_loc)
        
        pred_batch_v = ssvm.predict(batch_wifi_v)
        total_error = 0

        center_longitude_pred = 0
        center_latitude_pred = 0
        center_longitude_gt = 0
        center_latitude_gt = 0
        
        real_longitude = 0
        real_latitude = 0

        iter_test = 0
        
        window = np.zeros((seq_window))
        
        gp_total_error = 0
        gp_window = np.zeros((seq_window))
        
        for i in range(1,batch_num+1):
            for j in range(seq_window):
                print(pred_batch_v[i][j], batch_loc_v[i][j])
                print(batch_wifi_v[i][j].shape)
                
                ans = state_map_grid[pred_batch_v[i][j]][2].predict(batch_wifi_v[i][j].reshape(1,520))
                
                center_longitude_pred = (float(state_map_grid[pred_batch_v[i][j]][0])+0.5)*longitude_list[1]
                center_latitude_pred = (float(state_map_grid[pred_batch_v[i][j]][1])+0.5)*latitude_list[1]
                center_longitude_gt = (float(state_map_grid[batch_loc_v[i][j]][0])+0.5)*longitude_list[1]
                center_latitude_gt = (float(state_map_grid[batch_loc_v[i][j]][1])+0.5)*latitude_list[1]
                
                print(ans.shape)
                
                ans[0][0] += center_longitude_pred + longitude_list[2]
                ans[0][1] += center_latitude_pred + latitude_list[2]
                real_longitude = wifi_loc_time_v[iter_test][520]
                real_latitude = wifi_loc_time_v[iter_test][521]
                gp_total_error += math.sqrt(pow(ans[0][0] - real_longitude,2)+pow(ans[0][1] - real_latitude,2))
                gp_window[j] += math.sqrt(pow(ans[0][0] - real_longitude,2)+pow(ans[0][1] - real_latitude,2))
                
                iter_test+=1
                
                total_error += math.sqrt(pow(center_longitude_pred - center_longitude_gt,2)+pow(center_latitude_pred - center_latitude_gt,2))
                            
                window[j] += math.sqrt(pow(center_longitude_pred - center_longitude_gt,2)+pow(center_latitude_pred - center_latitude_gt,2))
                

        print("RMSE: ", total_error/(batch_num*seq_window))
        print("Windows:")
        for x in range(seq_window):
            print(" ", x, window[x]/batch_num)
        print("gp")
        print("RMSE: ", gp_total_error/(batch_num*seq_window))
        print("Windows:")
        for x in range(seq_window):
            print(" ", x, gp_window[x]/batch_num)
        print("Finish\n")

1443
On floor  2  in building  0 there are  2  users.
total_batch_num:  159
User 0  walking  733
31 19 0
31 19 (520,)
(2, 520)
31 19 (2, 520)
(3, 520)
31 19 (3, 520)
(4, 520)
31 19 (4, 520)
(5, 520)
31 19 (5, 520)
(6, 520)
31 19 (6, 520)
(7, 520)
31 19 (7, 520)
(8, 520)
31 19 (8, 520)
(9, 520)
31 19 (9, 520)
(10, 520)
32 17 1
32 17 (520,)
(2, 520)
32 17 (2, 520)
(3, 520)
32 17 (3, 520)
(4, 520)
32 17 (4, 520)
(5, 520)
32 17 (5, 520)
(6, 520)
32 17 (6, 520)
(7, 520)
32 17 (7, 520)
(8, 520)
32 17 (8, 520)
(9, 520)
32 17 (9, 520)
(10, 520)
33 14 2
33 14 (520,)
(2, 520)
33 14 (2, 520)
(3, 520)
33 14 (3, 520)
(4, 520)
33 14 (4, 520)
(5, 520)
33 14 (5, 520)
(6, 520)
33 14 (6, 520)
(7, 520)
33 14 (7, 520)
(8, 520)
33 14 (8, 520)
(9, 520)
33 14 (9, 520)
(10, 520)
33 13 3
33 13 (520,)
(2, 520)
33 13 (2, 520)
(3, 520)
33 13 (3, 520)
(4, 520)
33 13 (4, 520)
(5, 520)
33 13 (5, 520)
(6, 520)
33 13 (6, 520)
(7, 520)
33 13 (7, 520)
(8, 520)
33 13 (8, 520)
(9, 520)
33 13 (9, 520)
(10, 520)
34 10 4
34 

(9, 520)
68 51 (9, 520)
(10, 520)
64 50 43
64 50 (520,)
(2, 520)
64 50 (2, 520)
(3, 520)
64 50 (3, 520)
(4, 520)
64 50 (4, 520)
(5, 520)
64 50 (5, 520)
(6, 520)
64 50 (6, 520)
(7, 520)
64 50 (7, 520)
(8, 520)
64 50 (8, 520)
(9, 520)
64 50 (9, 520)
(10, 520)
63 50 44
63 50 (520,)
(2, 520)
63 50 (2, 520)
(3, 520)
63 50 (3, 520)
(4, 520)
63 50 (4, 520)
(5, 520)
63 50 (5, 520)
(6, 520)
63 50 (6, 520)
(7, 520)
63 50 (7, 520)
(8, 520)
63 50 (8, 520)
(9, 520)
63 50 (9, 520)
(10, 520)
59 49 45
59 49 (520,)
(2, 520)
59 49 (2, 520)
(3, 520)
59 49 (3, 520)
(4, 520)
59 49 (4, 520)
(5, 520)
59 49 (5, 520)
(6, 520)
59 49 (6, 520)
(7, 520)
59 49 (7, 520)
(8, 520)
59 49 (8, 520)
(9, 520)
59 49 (9, 520)
(10, 520)
58 49 46
58 49 (520,)
(2, 520)
58 49 (2, 520)
(3, 520)
58 49 (3, 520)
(4, 520)
58 49 (4, 520)
(5, 520)
58 49 (5, 520)
(6, 520)
58 49 (6, 520)
(7, 520)
58 49 (7, 520)
(8, 520)
58 49 (8, 520)
(9, 520)
58 49 (9, 520)
(10, 520)
49 48 47
49 48 (520,)
(2, 520)
49 48 (2, 520)
(3, 520)
49 48 (3, 520)


(22, 520)
33 65 (22, 520)
(23, 520)
33 65 (23, 520)
(24, 520)
33 65 (24, 520)
(25, 520)
33 65 (25, 520)
(26, 520)
33 65 (26, 520)
(27, 520)
32 69 (10, 520)
(11, 520)
32 69 (11, 520)
(12, 520)
32 69 (12, 520)
(13, 520)
32 69 (13, 520)
(14, 520)
32 69 (14, 520)
(15, 520)
32 69 (15, 520)
(16, 520)
32 69 (16, 520)
(17, 520)
32 69 (17, 520)
(18, 520)
32 69 (18, 520)
(19, 520)
32 69 (19, 520)
(20, 520)
33 68 (9, 520)
(10, 520)
33 68 (10, 520)
(11, 520)
33 68 (11, 520)
(12, 520)
33 68 (12, 520)
(13, 520)
33 68 (13, 520)
(14, 520)
33 68 (14, 520)
(15, 520)
33 68 (15, 520)
(16, 520)
33 68 (16, 520)
(17, 520)
33 68 (17, 520)
(18, 520)
33 68 (18, 520)
(19, 520)
33 68 (19, 520)
(20, 520)
33 68 (20, 520)
(21, 520)
33 68 (21, 520)
(22, 520)
33 68 (22, 520)
(23, 520)
33 68 (23, 520)
(24, 520)
33 68 (24, 520)
(25, 520)
33 68 (25, 520)
(26, 520)
33 68 (26, 520)
(27, 520)
34 65 (10, 520)
(11, 520)
34 65 (11, 520)
(12, 520)
34 65 (12, 520)
(13, 520)
34 65 (13, 520)
(14, 520)
34 65 (14, 520)
(15, 520)
34 

(19, 520)
29 16 (19, 520)
(20, 520)
23 23 (10, 520)
(11, 520)
23 23 (11, 520)
(12, 520)
23 23 (12, 520)
(13, 520)
23 23 (13, 520)
(14, 520)
23 23 (14, 520)
(15, 520)
23 23 (15, 520)
(16, 520)
23 23 (16, 520)
(17, 520)
23 23 (17, 520)
(18, 520)
23 23 (18, 520)
(19, 520)
23 23 (19, 520)
(20, 520)
21 24 (10, 520)
(11, 520)
21 24 (11, 520)
(12, 520)
21 24 (12, 520)
(13, 520)
21 24 (13, 520)
(14, 520)
21 24 (14, 520)
(15, 520)
21 24 (15, 520)
(16, 520)
21 24 (16, 520)
(17, 520)
21 24 (17, 520)
(18, 520)
21 24 (18, 520)
(19, 520)
19 23 (10, 520)
(11, 520)
19 23 (11, 520)
(12, 520)
19 23 (12, 520)
(13, 520)
19 23 (13, 520)
(14, 520)
19 23 (14, 520)
(15, 520)
19 23 (15, 520)
(16, 520)
19 23 (16, 520)
(17, 520)
19 23 (17, 520)
(18, 520)
19 23 (18, 520)
(19, 520)
19 23 (19, 520)
(20, 520)
16 22 (19, 520)
(20, 520)
16 22 (20, 520)
(21, 520)
16 22 (21, 520)
(22, 520)
16 22 (22, 520)
(23, 520)
16 22 (23, 520)
(24, 520)
16 22 (24, 520)
(25, 520)
16 22 (25, 520)
(26, 520)
16 22 (26, 520)
(27, 520)
16

(1, 2)
57 49
(520,)
(1, 2)
33 62
(520,)
(1, 2)
51 51
(520,)
(1, 2)
61 60
(520,)
(1, 2)
48 58
(520,)
(1, 2)
57 57
(520,)
(1, 2)
51 59
(520,)
(1, 2)
59 57
(520,)
(1, 2)
64 31
(520,)
(1, 2)
19 0
(520,)
(1, 2)
19 0
(520,)
(1, 2)
19 0
(520,)
(1, 2)
19 0
(520,)
(1, 2)
19 0
(520,)
(1, 2)
19 0
(520,)
(1, 2)
19 0
(520,)
(1, 2)
19 0
(520,)
(1, 2)
19 0
(520,)
(1, 2)
RMSE:  10.925126329529428
Windows:
  0 9.269911040929482
  1 8.518212466951669
  2 8.210118923663483
  3 13.549778117893588
  4 14.963662089736264
  5 10.867012026366151
  6 13.45642447837133
  7 9.87625342510156
  8 9.614764396751262
gp
RMSE:  11.05696420340149
Windows:
  0 10.41550518353809
  1 8.92725561385424
  2 9.706550090104088
  3 14.648815096300565
  4 15.772880965507047
  5 9.807092396931418
  6 11.791521819357236
  7 9.254933446755395
  8 9.188123218265288
Finish

